In [ ]:
import torch 
print(torch.__version__)

In [10]:
!nvidia-smi

Thu May 19 15:28:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |      2MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-1.11.0+cu113.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
!pip install ogb

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [14]:
import os
import random
import argparse
import numpy as np
import networkx as nx
import pandas as pd

import torch
import torch.nn.functional as F
import torch_sparse

from torch import Tensor
from torch.nn import Linear
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
import torch_geometric.nn.conv as ccgeo
from torch_geometric.nn.conv import MessagePassing


In [21]:
%%writefile main.py

from torch import Tensor
from torch.nn import Linear, ReLU
from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling, to_networkx
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, OptTensor, Size
from torch_sparse import SparseTensor, matmul
import torch_geometric.nn.conv as ccgeo
from torch_geometric.nn.conv import MessagePassing

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator
import pynvml as pynv
import os
import random
import argparse
import numpy as np
import networkx as nx
import pandas as pd

import torch
import torch.nn.functional as F
import torch_sparse

import math
##### SCRIPT STARTS HERE #####
#!usr/bin/bash python
filename="test.txt"


def get_spd_matrix_original(G, S, max_spd=5):
  print(len(G))
  spd_matrix = np.zeros((G.number_of_nodes(), len(S)), dtype=np.float32) # returns a 0-matrix of (4267, 500)
  for i, node_S in enumerate(S):                                          # i is index of for loop, node_S is the i-th element of S (current node)
      for node, length in nx.shortest_path_length(G, source=node_S).items():  # iterate over the Graph where source is node_S, let length denote the distance from node_S to current node
          spd_matrix[node, i] = min(length, max_spd)                          # refill element spd[node,i] with min(length, max_spd)  
  return spd_matrix

def get_spd_matrix_average_distance(G, S): # nx_graph is the complete graph networkx DS, node_subset is np.random.choice(nx_graph.number_of_nodes(), size=500, replace=False)
    # """S parameter draws (In the original version, get_spd_matrix is called in line ~363ish) 500 random samples within range 0 to G.number_of_nodes(),
    # (in one instance, this range is 0 to 4267)
    # """
    average = nx.average_shortest_path_length(G)
    spd_matrix = np.zeros((G.number_of_nodes(), len(S)), dtype=np.float32) # returns a 0-matrix of (4267, 500)
    for i, node_S in enumerate(S):                                          # i is index of for loop, node_S is the i-th element of S (current node)
        for node, length in nx.shortest_path_length(G, source=node_S).items():  # iterate over the Graph where source is node_S, let length denote the distance from node_S to current node
            spd_matrix[node, i] = length / average                 # refill element spd[node,i] with min(length, max_spd)  
    return spd_matrix

def get_similarity_spd_matrix(G, S, max_spd=5): # nx_graph is the complete graph networkx DS, node_subset is np.random.choice(nx_graph.number_of_nodes(), size=500, replace=False)
    # """S parameter draws (In the original version, get_spd_matrix is called in line ~363ish) 500 random samples within range 0 to G.number_of_nodes(),
    # (again , in one instance, this range is 0 to 4267)
    # """
    print(len(G))
    spd_matrix = np.zeros((G.number_of_nodes(), len(S)), dtype=np.float32) # returns a 0-matrix of (4267, 500)
    print("spd_for")
    for i, node_S in enumerate(S):                                          # i is index of for loop, node_S is the i-th element of S (current node)
        for node, length in nx.shortest_path_length(G, source=node_S).items():  # iterate over the Graph where source is node_S, let length denote the distance from node_S to current node
            spd_matrix[node, i] = min(length, max_spd)                          # refill element spd[node,i] with min(length, max_spd)  
            # print("spd_matrix[node, i]: ", spd_matrix[node, i])   
                                      
        for node, similarity in nx.simrank_similarity(G, source=node_S).items():  
            spd_matrix[node, i] = spd_matrix[node, i] + 150*similarity  
            # print("After similarity addition, spd_matrix[node, i]: ", spd_matrix[node, i])   
            # print("Similarity: ", 100*similarity)       
    
    print("spd_end")
    return spd_matrix



class Logger(object):
    def __init__(self, runs, info=None):
        self.info = info
        self.results = [[] for _ in range(runs)]

    def add_result(self, run, result):
        assert len(result) == 2
        assert run >= 0 and run < len(self.results)
        self.results[run].append(result)

    def print_statistics(self, run=None):
        global filename
        f = open(filename, "a")
        if run is not None:
            result = 100 * torch.tensor(self.results[run])
            x_df = pd.DataFrame(result)
            x_df.to_csv('tmp.csv')
            argmax = result[:, 0].argmax().item()
            print(f'Run {run + 1:02d}:', file=f)
            print(f'Highest Valid: {result[:, 0].max():.2f}', file=f)
            print(f'Highest Test: {result[:, 1].max():.2f}', file=f)
            print(f'   Final Test: {result[argmax, 1]:.2f}', file=f)
        else:
            result = 100 * torch.tensor(self.results)
            best_results = []
            for r in result:
                valid = r[:, 0].max().item()
                test = r[r[:, 0].argmax(), 1].item()
                best_results.append((valid, test))
            best_result = torch.tensor(best_results)
            print(f'All runs:', file=f)
            r = best_result[:, 0]
            print(f'Highest Valid: {r.mean():.4f} ± {r.std():.4f}', file=f)
            r = best_result[:, 1]
            print(f'   Final Test: {r.mean():.4f} ± {r.std():.4f}', file=f)
        f.close()

class SAGEConv(MessagePassing):
    r"""The GraphSAGE operator from the `"Inductive Representation Learning on
    Large Graphs" <https://arxiv.org/abs/1706.02216>`_ paper

    .. math::
        \mathbf{x}^{\prime}_i = \mathbf{W}_1 \mathbf{x}_i + \mathbf{W}_2 \cdot
        \mathrm{mean}_{j \in \mathcal{N(i)}} \mathbf{x}_j

    Args:
        in_channels (int or tuple): Size of each input sample. A tuple
            corresponds to the sizes of source and target dimensionalities.
        out_channels (int): Size of each output sample.
        normalize (bool, optional): If set to :obj:`True`, output features
            will be :math:`\ell_2`-normalized, *i.e.*,
            :math:`\frac{\mathbf{x}^{\prime}_i}
            {\| \mathbf{x}^{\prime}_i \|_2}`.
            (default: :obj:`False`)
        root_weight (bool, optional): If set to :obj:`False`, the layer will
            not add transformed root node features to the output.
            (default: :obj:`True`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.
    """
    def __init__(self, in_channels: Union[int, Tuple[int, int]],
                 out_channels: int, normalize: bool = False,
                 root_weight: bool = True,
                 bias: bool = True, **kwargs):  # yapf: disable
        kwargs.setdefault('aggr', 'mean')
        super(SAGEConv, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize
        self.root_weight = root_weight
        self.relu = ReLU(inplace=True)

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        self.lin_l = Linear(in_channels[0], out_channels, bias=bias)
        if self.root_weight:
            self.lin_r = Linear(in_channels[1], out_channels, bias=False)

        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()
        if self.root_weight:
            self.lin_r.reset_parameters()


    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                edge_attr: OptTensor = None, size: Size = None) -> Tensor:
        """"""
        if isinstance(x, Tensor):
            x: OptPairTensor = (x, x)

        # Node and edge feature dimensionalites need to match.
        if isinstance(edge_index, Tensor):
            assert edge_attr is not None
            assert x[0].size(-1) == edge_attr.size(-1)
        elif isinstance(edge_index, SparseTensor):
            assert x[0].size(-1) == edge_index.size(-1)

        # propagate_type: (x: OptPairTensor, edge_attr: OptTensor)
        out = self.propagate(edge_index, x=x, edge_attr=edge_attr, size=size)
        out = self.lin_l(out)

        x_r = x[1]
        if self.root_weight and x_r is not None:
            out += self.lin_r(x_r)

        if self.normalize:
            out = F.normalize(out, p=2., dim=-1)

        return out


    def message(self, x_j: Tensor, edge_attr: Tensor) -> Tensor:
        # all=x_j + edge_attr
        # all=all.cpu()
        # # all_list=list(torch.split(all, 1000000, dim=0)).cpu()
        # # print(len(all_list))

        # device = 'cuda:0'
        # device = torch.device(device)
        # return F.relu(all).to(device)
        return F.relu(x_j + edge_attr)


    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)


class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout):
        super(GraphSAGE,self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t, edge_attr, emb_ea):
        edge_attr = torch.mm(edge_attr, emb_ea)
        for conv in self.convs[:-1]:
            torch.cuda.empty_cache()
            x = conv(x, adj_t, edge_attr)
            torch.cuda.empty_cache()
            x = F.relu(x,inplace=True)
            torch.cuda.empty_cache()
            x = F.dropout(x, p=self.dropout, training=self.training)
            torch.cuda.empty_cache()
        x = self.convs[-1](x, adj_t, edge_attr)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(LinkPredictor, self).__init__()

        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.lins.append(torch.nn.Linear(hidden_channels, hidden_channels))
        self.lins.append(torch.nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()

    def forward(self, x_i, x_j):
        x = x_i * x_j
        for lin in self.lins[:-1]:
            x = lin(x)
            x = F.relu(x,inplace=True)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return torch.sigmoid(x)


def train(model, predictor, edge_attr, x, emb_ea, adj_t, split_edge, optimizer,device, batch_size):
    edge_index = adj_t
    model.train()
    predictor.train()
    pos_train_edge = split_edge

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size, shuffle=True,num_workers =4):
        # torch.cuda.empty_cache()
        optimizer.zero_grad()
        # h will be embeddings
        # x are the embeddings before
        # adj_t is edge index (All REAL edges)
        # edge_attr is x3 sampling of SPD
        # emb_ea is embedding of edges
        torch.cuda.empty_cache()
        
        h = model(x, adj_t, edge_attr, emb_ea)
        # torch.cuda.empty_cache()
        # print("Model output shape:",h.shape)
        # edge: we get a sampling of positive edge connections between nodes by using perm(list of nodes for the batch we know are positive)
        edge = pos_train_edge[perm].t()
        # we pass the edge list inside embedding h
        # we devide in edge[0] and edge[1] to devide the connected from->to
        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()
        # neg_sampling(we get negative edges from dataset) REAL ONES! they use the full dataset!!! VERY GOOD :)
        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='dense')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples


@torch.no_grad()
def test(model, predictor, edge_attr, x, emb_ea, adj_t, split_edge, evaluator, batch_size):
    model.eval()
    predictor.eval()

    h = model(x, adj_t, edge_attr, emb_ea)

    pos_valid_edge = split_edge['valid']['edge'].to(x.device)
    neg_valid_edge = split_edge['valid']['edge_neg'].to(x.device)
    pos_test_edge = split_edge['test']['edge'].to(x.device)
    neg_test_edge = split_edge['test']['edge_neg'].to(x.device)

    pos_valid_preds = []
    for perm in DataLoader(range(pos_valid_edge.size(0)), batch_size):
        edge = pos_valid_edge[perm].t()
        pos_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_valid_pred = torch.cat(pos_valid_preds, dim=0)

    neg_valid_preds = []
    for perm in DataLoader(range(neg_valid_edge.size(0)), batch_size):
        edge = neg_valid_edge[perm].t()
        neg_valid_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_valid_pred = torch.cat(neg_valid_preds, dim=0)

    pos_test_preds = []
    for perm in DataLoader(range(pos_test_edge.size(0)), batch_size):
        edge = pos_test_edge[perm].t()
        pos_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    pos_test_pred = torch.cat(pos_test_preds, dim=0)

    neg_test_preds = []
    for perm in DataLoader(range(neg_test_edge.size(0)), batch_size):
        edge = neg_test_edge[perm].t()
        neg_test_preds += [predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
    neg_test_pred = torch.cat(neg_test_preds, dim=0)

    results = {}
    for K in [20, 50, 100]:
        evaluator.K = K
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']

        results[f'Hits@{K}'] = (valid_hits, test_hits)

    return results

def print_gpu_utilization(position):
    print(position)
    os.system("nvidia-smi")
    # print(torch.cuda.memory_summary(device=None, abbreviated=False))


def main():
    cuda_device = '0'
    print('Done')
    global filename
    parser = argparse.ArgumentParser(description='Link_Pred_DDI')
    parser.add_argument('--device', type=str, default="cuda:0")
    parser.add_argument('--num_layers', type=int, default=2)
    parser.add_argument('--num_samples', type=int, default=5)
    parser.add_argument('--node_emb', type=int, default=256)
    parser.add_argument('--hidden_channels', type=int, default=256)
    parser.add_argument('--dropout', type=float, default=0.3)
    parser.add_argument('--spd_size', type=int, default=500)
    # parser.add_argument('--batch_size', type=int, default=1) 
    # TODO
    parser.add_argument('--batch_size', type=int, default=64 * 1024)
    parser.add_argument('--lr', type=float, default=0.003)
    parser.add_argument('--epochs', type=int, default=400)
    parser.add_argument('--log_steps', type=int, default=1)
    parser.add_argument('--eval_steps', type=int, default=1)
    parser.add_argument('--runs', type=int, default=10)

    parser.add_argument('--type', type=str, default="original")

    args = parser.parse_args()
    print(args)
    device = args.device if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    

    dataset = PygLinkPropPredDataset(name='ogbl-ddi')
    data = dataset[0]
    print(data.size())
    edge_index = data.edge_index.to(device)
    print_gpu_utilization("Start")

    split_edge = dataset.get_edge_split()

    model = GraphSAGE(args.node_emb, args.hidden_channels, args.hidden_channels,
                      args.num_layers, args.dropout).to(device)

    emb = torch.nn.Embedding(data.num_nodes, args.node_emb).to(device)
    emb_ea = torch.nn.Embedding(args.num_samples, args.node_emb).to(device)
    predictor = LinkPredictor(args.hidden_channels, args.hidden_channels, 1,
                              args.num_layers+1, args.dropout).to(device)

    print('Number of parameters:',
          sum(p.numel() for p in list(model.parameters()) +
          list(predictor.parameters()) + list(emb.parameters()) + list(emb_ea.parameters())))

    # encode distance information
    np.random.seed(0)
    nx_graph = to_networkx(data, to_undirected=True)
    node_mask = []
    spd_size=args.spd_size
    for _ in range(args.num_samples):
        node_mask.append(np.random.choice(spd_size, size=int(np.round(spd_size/2.5)), replace=False))
    node_mask = np.array(node_mask)

    



    type_exec=args.type

    original_subset = np.random.choice(nx_graph.number_of_nodes(), size=spd_size, replace=False)
    if(type_exec=="original"):
      print("Original method selected")
      filename="./gdrive/MyDrive/output_original.txt"
      spd = get_spd_matrix_original(G=nx_graph, S=original_subset, max_spd=5)
    elif(type_exec=="average"):
      print("Average method selected")
      filename="./gdrive/MyDrive/output_average.txt"
      spd = get_spd_matrix_average_distance(G=nx_graph, S=original_subset)
    elif(type_exec=="top-k"):
      print("top-k method selected")
      filename="./gdrive/MyDrive/output_topk.txt"
      S = sorted(nx_graph.degree, key=lambda x: x[1], reverse=True)
      S = S[0:spd_size]
      s_nodes = np.zeros(len(S))
      for i in range(len(S)):
          s_nodes[i] = S[i][0] 
      s_nodes = s_nodes.astype(int)
      spd = get_spd_matrix_original(G=nx_graph, S=s_nodes, max_spd=5)
    elif(type_exec=="similarity"):
      print("Similarity method selected")
      filename="./gdrive/MyDrive/output_similarity.txt"
      spd = get_similarity_spd_matrix(G=nx_graph, S=original_subset, max_spd=5)
    else:
      print("Original method auto-chosen")
      filename="./gdrive/MyDrive/output_original.txt"
      spd = get_spd_matrix_original(G=nx_graph, S=original_subset, max_spd=5)

    torch.cuda.empty_cache()
    spd = torch.Tensor(spd).to(device)
    print_gpu_utilization("spd loaded")
    edge_attr = spd[edge_index, :].mean(0)[:, node_mask].mean(2)
    a_max = torch.max(edge_attr, dim=0, keepdim=True)[0]
    a_min = torch.min(edge_attr, dim=0, keepdim=True)[0]
    edge_attr = (edge_attr - a_min) / (a_max - a_min + 1e-6)
    evaluator = Evaluator(name='ogbl-ddi')
    loggers = {
        'Hits@20': Logger(args.runs, args),
        'Hits@50': Logger(args.runs, args),
        'Hits@100': Logger(args.runs, args),
    }
    for run in range(args.runs):
        torch.cuda.empty_cache()
        # print_gpu_utilization("start run cache cleared")
        random.seed(run)
        torch.manual_seed(run)
        torch.nn.init.xavier_uniform_(emb.weight)
        torch.nn.init.xavier_uniform_(emb_ea.weight)
        model.reset_parameters()
        predictor.reset_parameters()
        optimizer = torch.optim.Adam(
            list(model.parameters()) + list(emb.parameters()) +
            list(emb_ea.parameters()) + list(predictor.parameters()), lr=args.lr)
        split_edge_in=split_edge['train']['edge'].to(device)
        for epoch in range(1, 1 + args.epochs):
            torch.cuda.empty_cache()
            # print_gpu_utilization("in epoch cache cleared")
            loss = train(model, predictor, edge_attr, emb.weight, emb_ea.weight, edge_index, split_edge_in,
                         optimizer,device, args.batch_size)
            torch.cuda.empty_cache()
            # print_gpu_utilization("after pass cache cleared")
            if epoch % args.eval_steps == 0:
                results = test(model, predictor, edge_attr, emb.weight, emb_ea.weight, edge_index, split_edge,
                               evaluator, args.batch_size)
                for key, result in results.items():
                    loggers[key].add_result(run, result)

                if epoch % args.log_steps == 0:
                    f = open(filename, "a")
                    for key, result in results.items():
                        valid_hits, test_hits = result
                        print(key, file=f)
                        print(f'Run: {run + 1:02d}, '
                              f'Epoch: {epoch:02d}, '
                              f'Loss: {loss:.4f}, '
                              f'Valid: {100 * valid_hits:.2f}%, '
                              f'Test: {100 * test_hits:.2f}%', file=f)
                    print('---', file=f)
                    f.close()
        torch.save(model,"save.pth")
        for key in loggers.keys():
            print(key)
            loggers[key].print_statistics(run)

    for key in loggers.keys():
        print(key)
        loggers[key].print_statistics()

if __name__ == "__main__":
    main()

Overwriting main.py


In [ ]:
original
average
top-k
similarity

In [ ]:
# Run Training (Original code)
!python3 main.py --node_emb 254 --hidden_channels 254 --num_samples 3 --num_layers 2 --epochs 1000 --spd_size 500 --runs 1 --type original

In [ ]:
# Run Training (Average code)
!python3 main.py --node_emb 254 --hidden_channels 254 --num_samples 3 --num_layers 2 --epochs 1000 --spd_size 500 --runs 1 --type average

In [ ]:
# Run Training (Top-k code)
!python3 main.py --node_emb 254 --hidden_channels 254 --num_samples 3 --num_layers 2 --epochs 1000 --spd_size 500 --runs 1 --type top-k

In [ ]:
# Run Training (similarity code)
!python3 main.py --node_emb 254 --hidden_channels 254 --num_samples 3 --num_layers 2 --epochs 1000 --spd_size 500 --runs 1 --type similarity